In [1]:
import os
import sys
import numpy as np
sys.path.append("/Users/atrask/Laboratory/clones/pycolab")
import curses
import torch
import six
import itertools
import collections

from campx import things
from campx.ascii_art import ascii_art_to_game, Partial
from campx import engine

In [9]:
class  RollingDrape(things.Drape):
  """A Drape that just `np.roll`s the mask around either axis."""

  # There are four rolls to choose from: two shifts of size 1 along both axes.
  _ROLL_AXES = [0, 0, 1, 1]
  _ROLL_SHIFTS = [-1, 1, -1, 1]

  def update(self, actions, board, layers, backdrop, all_things, the_plot):
    del board, layers, backdrop, all_things  # unused

    if actions is None: return  # No work needed to make the first observation.
    if actions == 4: the_plot.terminate_episode()  # Action 4 means "quit".

    # If the player has chosen a motion action, use that action to index into
    # the set of four rolls.
    if actions < 4:
      rolled = np.roll(self.curtain.numpy(),  # Makes a copy, alas.
                       self._ROLL_SHIFTS[actions], self._ROLL_AXES[actions])
      rolled = torch.ByteTensor(rolled)
      self.curtain.set_(rolled)
      the_plot.add_reward(1)  # Give ourselves a point for moving.

class SlidingSprite(things.Sprite):
  """A Sprite that moves in diagonal directions."""

  # We have four mappings from actions to motions to choose from. The mappings
  # are arranged so that given any index i, then across all sets, the motion
  # that undoes motion i always has the same index j.
  _DX = ([-1, 1, -1, 1], [-1, 1, -1, 1], [1, -1, 1, -1], [1, -1, 1, -1])
  _DY = ([-1, 1, 1, -1], [1, -1, -1, 1], [1, -1, -1, 1], [-1, 1, 1, -1])

  def __init__(self, corner, position, character, direction_set):
    """Build a SlidingSprite.
    Args:
      corner: required argument for Sprite.
      position: required argument for Sprite.
      character: required argument for Sprite.
      direction_set: an integer in `[0,3]` that selects from any of four
          mappings from actions to (diagonal) motions.
    """
    super(SlidingSprite, self).__init__(corner, position, character)
    self._dx = self._DX[direction_set]
    self._dy = self._DY[direction_set]

  def update(self, actions, board, layers, backdrop, all_things, the_plot):
    del board, layers, backdrop, all_things, the_plot  # unused
    # Actions 0-3 are motion actions; the others we ignore.
    if actions is None or actions > 3: return
    new_col = (self._position.col + self._dx[actions]) % self.corner.col
    new_row = (self._position.row + self._dy[actions]) % self.corner.row
    self._position = self.Position(new_row, new_col)

In [10]:

HELLO_ART  = ['                                    ',
             '  #   #  ### #    #     ###         ',
             '  #   # #    #    #    #   #        ',
             '  ##### #### #    #    #   #        ',
             '  #   # #    #    #    #   #        ',
             '  #   #  ###  ###  ###  ###         ',
             '                                    ',
             '     @   @  @@@   @@@  @    @@@@  1 ',
             '     @   @ @   @ @   @ @    @   @ 2 ',
             '     @ @ @ @   @ @@@@  @    @   @ 3 ',
             '     @ @ @ @   @ @   @ @    @   @   ',
             '      @@@   @@@  @   @  @@@ @@@@  4 ',
             '                                    ']

def make_game():
  """Builds and returns a Hello World game."""
  return ascii_art_to_game(
      HELLO_ART,
      what_lies_beneath=' ',
      sprites={'1': Partial(SlidingSprite, 0),
               '2': Partial(SlidingSprite, 1),
               '3': Partial(SlidingSprite, 2),
               '4': Partial(SlidingSprite, 3)},
      drapes={'@': RollingDrape},
      z_order='12@34')
game = make_game()

In [11]:
board, reward, discount = game.its_showtime()

In [14]:
# this moves all the 64s upward each step
board, reward, discout = game.play(0)

board.board



Columns 0 to 12 
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0    64     0     0     0    64     0     0    64
    0     0     0     0     0    64     0     0     0    64     0    64     0
    0     0     0     0     0    64     0    64     0    64     0    64     0
    0     0     0     0     0    64     0    64     0    64     0    64     0
   52     0     0     0     0     0    64    64    64     0     0     0    64
    0     0     0     0     0     0     0     0     0     0     0     0     0
   51     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0    

In [13]:
reward

1